# Cell 1: Imports & Global Setup

In [9]:
# Standard Library Imports
import os
import re
import csv
from datetime import time as dt_time, timedelta

# Third-Party Imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats as stats
import scipy.signal as signal
from scipy.special import softmax
from scipy.signal import welch, butter, filtfilt, medfilt, find_peaks
from scipy.interpolate import interp1d, CubicSpline
from scipy.ndimage import median_filter
from tqdm.auto import tqdm
import neurokit2 as nk
import yaml
from scipy.signal import resample_poly
from fractions import Fraction
# Logging Configuration (place this after imports, at the very top)
import os
import logging
from datetime import datetime


# Load config
with open('D:/PhD_Projects/TSST_Stress_Workload_Pipeline/scripts/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Access relevant paths from config
data_path = config["paths"]["raw_data"]
output_path = config["paths"]["output"]
subjective_file_path = os.path.join(config["paths"]["output"], "aggregated", "subjective.csv")



# Enable tqdm for pandas
tqdm.pandas()

# --- Create logs directory if it doesn't exist ---
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# --- Create timestamped log file name ---
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = os.path.join(log_dir, f"data_processing_{timestamp}.log")

# --- Configure logging ---
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)

# QC Logger Setup: One logger per participant
qc_loggers = {}

for participant_id in range(1, 49):
    qc_log_path = os.path.join(config['paths']['physio_qc'], f'{participant_id}_qc.txt')
    os.makedirs(os.path.dirname(qc_log_path), exist_ok=True)

    logger = logging.getLogger(f'qc_logger_{participant_id}')
    logger.setLevel(logging.INFO)

    # Avoid duplicate handlers if re-running
    if not logger.handlers:
        handler = logging.FileHandler(qc_log_path, mode='a')
        formatter = logging.Formatter('%(asctime)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.propagate = False

    qc_loggers[participant_id] = logger




# Cell 2: Constants & Definitions

In [ ]:


def extract_gsr_features(signal_data, sampling_rate=10,
                         min_scr_amp=0.03, max_scr_amp=3.0,
                         flat_std_thresh=0.05):
    """
    Extract tonic and phasic EDA/GSR features with built-in QC.

    Tonic features are always returned.
    Phasic (SCR) features are set to NaN if the segment is "flat".

    Parameters
    ----------
    signal_data : array-like
        Cleaned skin-conductance signal (µS).
    sampling_rate : int, optional
        Sampling frequency of signal_data (Hz). Default 10.
    min_scr_amp, max_scr_amp : float, optional
        Amplitude thresholds (µS) to define a "meaningful" SCR.
    flat_std_thresh : float, optional
        Standard-deviation threshold (µS) below which phasic
        features are considered unreliable.

    Returns
    -------
    dict
        Dictionary of tonic features, phasic features (or NaN if flat),
        and a LowVar_Flag indicator.
    """
    base_keys = [
        "EDA_Tonic_Mean","EDA_Tonic_SD","EDA_PeakRate",
        "EDA_PeakHeight_Mean","EDA_PeakHeight_Max",
        "EDA_PeakHeight_Median","EDA_PeakArea",
        "EDA_TotalSCRs","EDA_MeaningfulSCRs",
        "EDA_ProportionMeaningful","LowVar_Flag"
    ]

    if len(signal_data) < 10:
        # too short for reliable stats
        return {k: np.nan for k in base_keys}

    try:
        eda_signals, info = nk.eda_process(
            signal_data,
            sampling_rate=sampling_rate,
            method="neurokit"
        )

        # ---------- tonic ----------
        tonic = eda_signals["EDA_Tonic"].dropna()
        tonic_mean = float(np.nanmean(tonic))
        tonic_sd   = float(np.nanstd(tonic))

        # --- check for flat segment ---
        lowvar_flag = bool(tonic_sd < flat_std_thresh)

        # ---------- phasic ----------
        if lowvar_flag:
            # Segment too flat: keep tonic, mask phasic
            phasic_feats = {k: np.nan for k in [
                "EDA_PeakRate","EDA_PeakHeight_Mean","EDA_PeakHeight_Max",
                "EDA_PeakHeight_Median","EDA_PeakArea",
                "EDA_TotalSCRs","EDA_MeaningfulSCRs","EDA_ProportionMeaningful"
            ]}
        else:
            scr_amplitudes = eda_signals["SCR_Amplitude"].dropna()
            scr_peaks      = eda_signals["SCR_Peaks"].sum()
            duration_sec   = len(signal_data) / sampling_rate
            peak_rate      = scr_peaks / duration_sec if duration_sec > 0 else np.nan

            phasic_feats = {
                "EDA_PeakRate": peak_rate,
                "EDA_PeakHeight_Mean": float(np.nanmean(scr_amplitudes)),
                "EDA_PeakHeight_Max":  float(np.nanmax(scr_amplitudes)),
                "EDA_PeakHeight_Median": float(np.nanmedian(scr_amplitudes)),
                "EDA_PeakArea": float(np.nansum(scr_amplitudes)),
                "EDA_TotalSCRs": int(scr_peaks),
                "EDA_MeaningfulSCRs": int(
                    ((scr_amplitudes >= min_scr_amp) &
                     (scr_amplitudes <= max_scr_amp)).sum()
                ),
                "EDA_ProportionMeaningful": (
                    ((scr_amplitudes >= min_scr_amp) &
                     (scr_amplitudes <= max_scr_amp)).mean()
                    if len(scr_amplitudes) > 0 else np.nan
                )
            }

        return {
            "EDA_Tonic_Mean": tonic_mean,
            "EDA_Tonic_SD":   tonic_sd,
            **phasic_feats,
            "LowVar_Flag":    lowvar_flag
        }

    except Exception as e:
        logging.warning(f"[GSR] Feature extraction error: {e}")
        return {k: np.nan for k in base_keys}
    

def pupil_unrest(left, right, fs=30, band=(0.05,0.3)):
    mask = np.isfinite(left) & np.isfinite(right)
    if mask.sum() < fs*5: 
        return np.nan
    f, Pxx = welch((left[mask]+right[mask])/2,
                   fs=fs, nperseg=fs*30, scaling="density")
    sel = (f >= band[0]) & (f <= band[1])
    return np.trapz(Pxx[sel], f[sel])



# Function to calculate statistics for a time window before an event
#def calculate_stats(row, data, columns, filter_value=None, context="Full"):
def calculate_stats(data, columns, participant_id, condition, filter_value=None, context="Full"):
    all_stats = {}

   # participant_id = row.get('Participant_ID', 'Unknown')
   # condition = row.get('Condition', 'Unknown')

    columns = [col for col in columns if isinstance(col, str)]

    for column in columns:
        if not isinstance(column, str):
            print(f"[DEBUG] Unexpected non-string column in calculate_stats: {column} ({type(column)})")
            continue

        col_data = data[column].dropna()

        if filter_value is not None:
            col_data = col_data[col_data != filter_value]

        # --- Compute basic stats ---
        #Set limit to 0 to debug missing data
        if len(col_data) < 0 * len(data[column]):
            missing_percentage = (1 - (len(col_data) / len(data[column]))) * 100
            logging.info(f"[STATS]-{context.upper()}] Participant {participant_id}, Condition {condition}, Column '{column}': "
                         f"Missing data = {missing_percentage:.2f}% (>20%), basic stats set to NaN.")

            stats = {
                f"{column}_Median": np.nan,
                f"{column}_Mean": np.nan,
                f"{column}_SD": np.nan,
                f"{column}_MIN": np.nan,
                f"{column}_MAX": np.nan
            }


        else:
            stats = {
                f"{column}_Median": col_data.median() if not col_data.empty else np.nan,
                f"{column}_Mean": col_data.mean() if not col_data.empty else np.nan,
                f"{column}_SD": col_data.std() if not col_data.empty else np.nan,
                f"{column}_MIN": col_data.min() if not col_data.empty else np.nan,
                f"{column}_MAX": col_data.max() if not col_data.empty else np.nan
            }

        # --- pupil unrest power (compute once) ---
        if {"Foveal_Corrected_Dilation_Left_CLEANED_ABS",
            "Foveal_Corrected_Dilation_Right_CLEANED_ABS"} <= set(data.columns):
            unrest_val = pupil_unrest(
                data["Foveal_Corrected_Dilation_Left_CLEANED_ABS"],
                data["Foveal_Corrected_Dilation_Right_CLEANED_ABS"],
                fs=30
            )
            all_stats["Pupil_UnrestPower"] = unrest_val



        # ---------- detailed GSR features ----------
        if column == "Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS_CLEANED_NK":
            try:
                gsr_features = extract_gsr_features(
                    signal_data=np.asarray(col_data, dtype=float),
                    sampling_rate=10
                )

                # If the segment is essentially flat, blank only the phasic features
                too_flat = (stats[f"{column}_SD"] is not np.nan and
                            stats[f"{column}_SD"] < 0.05)

                for feat_name, val in gsr_features.items():
                    # Keep tonic metrics regardless; mask phasic if too flat
                    if too_flat and feat_name.startswith("EDA_Peak"):
                        stats[f"{column}_{feat_name}"] = np.nan
                    else:
                        stats[f"{column}_{feat_name}"] = val

            except Exception as e:
                logging.warning(
                    f"[GSR] Feature extraction failed "
                    f"(P{participant_id}, {condition}): {e}"
                )
                for feat_name in [
                    "EDA_Tonic_Mean","EDA_Tonic_SD","EDA_PeakRate",
                    "EDA_PeakHeight_Mean","EDA_PeakHeight_Max",
                    "EDA_PeakHeight_Median","EDA_PeakArea",
                    "EDA_TotalSCRs","EDA_MeaningfulSCRs",
                    "EDA_ProportionMeaningful"
                ]:
                    stats[f"{column}_{feat_name}"] = np.nan

        all_stats.update(stats)

    return all_stats


def extract_hrv_features(rr_intervals_ms):
    print("🔍 extract_hrv_features called")
    
    # 1) Clean up
    rr = np.asarray(rr_intervals_ms, dtype=float)
    print(f"1) Raw input length: {len(rr)}")
    rr = rr[~np.isnan(rr)]
    print(f"   After NaN removal: {len(rr)} values; head/tail: {rr[:5]} ... {rr[-5:]}")
    if len(rr) < 2:
        print("   Not enough data for HRV. Returning NaNs.")
        return pd.DataFrame([{"HRV_RMSSD": np.nan,
                              "HRV_SDNN":  np.nan,
                              "HRV_pNN50": np.nan}])
    
    # 2) Show basic stats
    print(f"2) RR stats (ms): min={rr.min()}, max={rr.max()}, mean={rr.mean():.2f}, std={rr.std():.2f}")
    
    # 3) Convert to seconds
    rr_s = rr / 1000.0
    print(f"3) Converted to seconds; head/tail: {rr_s[:5]} ... {rr_s[-5:]}")
    
    # 4) Build the RRI dict for NeuroKit2
    rri_time = np.cumsum(rr_s)
    rri_dict = {"RRI": rr_s, "RRI_Time": rri_time}
    print(f"4) RRI dict prepared; first 5 times: {rri_time[:5]}")
    
    # 5) Compute time-domain HRV
    print("5) Computing time-domain HRV with nk.hrv_time()")
    hrv_td = nk.hrv_time(rri_dict, sampling_rate=None, show=False)
    print("   HRV DataFrame:")
    print(hrv_td.head())
    
    # 6) Extract and return metrics
    metrics = hrv_td[["HRV_RMSSD", "HRV_SDNN", "HRV_pNN50"]].reset_index(drop=True)
    print("6) Returning metrics:")
    print(metrics)
    return metrics



def extract_head_velocity_features(df, time_col="Headset-TimeStamp", max_speed=2.0):
    """Add 'Head_Velocity': 3-D translational speed (m/s), 
       ignoring non-monotonic time and implausible teleports."""
    df = df.sort_values(time_col).drop_duplicates(time_col).copy()

    t = df[time_col].to_numpy()
    dt = np.diff(t)
    dt[dt <= 0] = np.nan

    dx = np.diff(df["Headset-pX"])
    dy = np.diff(df["Headset-pY"])
    dz = np.diff(df["Headset-pZ"])
    speed = np.sqrt(dx**2 + dy**2 + dz**2) / dt

    # clip or mask unrealistic jumps
    speed[speed > max_speed] = np.nan

    df["Head_Velocity"] = np.concatenate(([np.nan], speed))
    return df



def extract_response_metrics(window_df):
    """
    Extracts response metrics from a given window of data.
    Returns counts, latency, and accuracy information.
    """
    response_rows = window_df[window_df["Response"].notna()]
    response_count = len(response_rows)

    # duration in seconds (float)
    duration_sec = (
        window_df["Time_From_Start_Seconds"].max()
        - window_df["Time_From_Start_Seconds"].min()
    )

    # If the column is datetime/timedelta convert to seconds;
    # if it’s already numeric this is a no-op
    if pd.api.types.is_timedelta64_dtype(duration_sec):
        duration_sec = duration_sec.total_seconds()
    elif hasattr(duration_sec, "total_seconds"):  # single Timedelta object
        duration_sec = duration_sec.total_seconds()

    duration_sec = duration_sec if duration_sec > 0 else np.nan

    response_rate_per_min = (
        (response_count / duration_sec) * 60 if pd.notna(duration_sec) else np.nan
    )
    mean_response_latency = (
        response_rows["Response_Time"].mean() if response_count > 0 else np.nan
    )

    correct_count = (response_rows["Response"] == "Correct").sum()
    incorrect_count = (response_rows["Response"] == "Incorrect").sum()
    accuracy = correct_count / response_count if response_count > 0 else np.nan

    usable = response_count >= 3

    return {
        "Response_Count": response_count,
        "Response_Rate_per_min": response_rate_per_min,
        "Mean_Response_Latency_sec": mean_response_latency,
        "Response_Accuracy": accuracy,
        "Response_Correct": correct_count,
        "Response_Incorrect": incorrect_count,
        "Response_Usable": usable,
    }





In [11]:
# Single Cell for Shared Constants and Definitions

# Define statistics categories
STATS_BASIC = ['Median', 'Mean', 'SD', 'MIN', 'MAX']
STATS_HRV = ['RMSSD', 'LF/HF_Ratio']
STATS_EDA = ['Peak_Rate', 'Peak_Height_Mean', 'Peak_Height_Max']

#### Calibration Measures
data_header_calibration_measures = ['Calibration_Median_Resting_HeartRate','Calibration_Mean_Resting_HeartRate','Calibration_SD_Resting_HeartRate','Calibration_Median_Resting_HeartRate_RR_Interval','Calibration_Mean_Resting_HeartRate_RR_Interval','Calibration_SD_Resting_HeartRate_RR_Interval',
'Calibration_Median_Resting_HeartRate_RR_Interval_CLEANED_ABS_Threshold','Calibration_Mean_Resting_HeartRate_RR_Interval_CLEANED_ABS_Threshold','Calibration_SD_Resting_HeartRate_RR_Interval_CLEANED_ABS_Threshold','Calibration_Median_Resting_HeartRate_RR_Interval_CLEANED_REL_Threshold',
'Calibration_Mean_Resting_HeartRate_RR_Interval_CLEANED_REL_Threshold','Calibration_SD_Resting_HeartRate_RR_Interval_CLEANED_REL_Threshold','Calibration_RR_Interval_RAW_RMSSD','Calibration_RR_Interval_CLEANED_ABS_RMSSD','Calibration_RR_Interval_CLEANED_REL_RMSSD',
'Calibration_Median_GSR_Conductance','Calibration_Mean_GSR_Conductance','Calibration_SD_GSR_Conductance','Calibration_Median_GSR_Resistance','Calibration_Mean_GSR_Resistance','Calibration_SD_GSR_Resistance','Calibration_Median_GSR_Conductance_CLEANED_ABS_Threshold',
'Calibration_Mean_GSR_Conductance_CLEANED_ABS_Threshold','Calibration_SD_GSR_Conductance_CLEANED_ABS_Threshold','Calibration_Median_GSR_Resistance_CLEANED_ABS_Threshold','Calibration_Mean_GSR_Resistance_CLEANED_ABS_Threshold','Calibration_SD_GSR_Resistance_CLEANED_ABS_Threshold',
'Calibration_Median_BlinkDuration','Calibration_Mean_BlinkDuration','Calibration_SD_BlinkDuration','Calibration_Median_InterBlinkInterval','Calibration_Mean_InterBlinkInterval','Calibration_SD_InterBlinkInterval','Calibration_PupilDilation_Left_0','Calibration_PupilDilation_Right_0','Calibration_PupilDilation_Left_16','Calibration_PupilDilation_Right_16','Calibration_PupilDilation_Left_32',
'Calibration_PupilDilation_Right_32','Calibration_PupilDilation_Left_48','Calibration_PupilDilation_Right_48','Calibration_PupilDilation_Left_64','Calibration_PupilDilation_Right_64','Calibration_PupilDilation_Left_80','Calibration_PupilDilation_Right_80','Calibration_PupilDilation_Left_96','Calibration_PupilDilation_Right_96',
'Calibration_PupilDilation_Left_112','Calibration_PupilDilation_Right_112','Calibration_PupilDilation_Left_128','Calibration_PupilDilation_Right_128','Calibration_PupilDilation_Left_144','Calibration_PupilDilation_Right_144','Calibration_PupilDilation_Left_160','Calibration_PupilDilation_Right_160','Calibration_PupilDilation_Left_176',
'Calibration_PupilDilation_Right_176','Calibration_PupilDilation_Left_192','Calibration_PupilDilation_Right_192','Calibration_PupilDilation_Left_208','Calibration_PupilDilation_Right_208','Calibration_PupilDilation_Left_224','Calibration_PupilDilation_Right_224','Calibration_PupilDilation_Left_240','Calibration_PupilDilation_Right_240',
'Calibration_PupilDilation_Left_255','Calibration_PupilDilation_Right_255']


# Define column names
column_names = [
    'Pupil_Dilation_Left', 'Pupil_Dilation_Right', 'Current_Blink_Duration',
    'Inter_Blink_Interval', 'Shimmer_D36A_GSR_Skin_Conductance_uS',
    'Shimmer_D36A_GSR_Skin_Resistance_kOhms', 'Polar_HeartRate_BPM',
    'Polar_HeartRate_RR_Interval', 'Foveal_Corrected_Dilation_Left', 'Foveal_Corrected_Dilation_Right', 'Response', 'Response_Time', 'Headset-pX', 'Headset-pY', 'Headset-pZ', 'Headset-TimeStamp'
]

range_columns = ['Current_Blink_Duration', 'Shimmer_D36A_GSR_Skin_Conductance_uS',
                 'Shimmer_D36A_GSR_Skin_Resistance_kOhms', 'Polar_HeartRate_RR_Interval']


ranges = [[50, 700], [0, None], [0, None], [200, 2000]]

# Define columns to process
columns_to_clean = [
    'Polar_HeartRate_RR_Interval',
    'Shimmer_D36A_GSR_Skin_Conductance_uS',
    'Shimmer_D36A_GSR_Skin_Resistance_kOhms',
    'Polar_HeartRate_BPM'
]





# Cell 3: Data Loading & Cleaning

In [12]:


# --------------------------------------------------------------------------
# Optional: Only fix headers & trailing commas if files haven't been cleaned
#           You can remove or comment these if your files are already stable.
# --------------------------------------------------------------------------
def fix_headers_and_commas():
    for root, _, files in os.walk(data_path):
        for filename in files:
            # Fix headers
            if filename.endswith('_Calibration_Data_RAW.csv') and 'BLINK' not in filename:
                file_path = os.path.join(root, filename)
                with open(file_path, 'r') as file:
                    data = file.read().replace(
                        'current_InterBlinkIntervalShimmer_D36A_GSR_Skin_Conductance_uS',
                        'current_InterBlinkInterval,Shimmer_D36A_GSR_Skin_Conductance_uS'
                    )
                with open(file_path, 'w') as file:
                    file.write(data)

def clean_csv_trailing_commas(file_path):
    temp_file = file_path + '.tmp'
    with open(file_path, 'r', newline='') as infile, open(temp_file, 'w', newline='') as outfile:
        reader, writer = csv.reader(infile), csv.writer(outfile)
        for row in reader:
            writer.writerow(row[:-1] if row and row[-1] == '' else row)
    os.replace(temp_file, file_path)

def clean_all_csv_trailing_commas(folder_path):
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('_Calibration_Data_RAW.csv') and 'BLINK' not in filename:
                clean_csv_trailing_commas(os.path.join(root, filename))
# --------------------------------------------------------------------------

def load_csv_files(folder_path, include_text, exclude_text=""):
    df_list = []
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.csv') and include_text in filename and exclude_text not in filename:
                file_path = os.path.join(root, filename)
                print(f"[INFO] Reading: {file_path}")
                df = pd.read_csv(file_path, index_col=False)

                # --- Rename the misspelled column if it exists ---
                if "Polar_HeartRate_BPM" in df.columns:
                    df.rename(columns={"Polar_HeartRate_BPM": "Polar_HeartRate_BPM"}, inplace=True)

                df_list.append(df)
    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()

def load_raw_data(folder_path, filename_filter):
    df_list = []

    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.csv') and filename_filter in filename:
                file_path = os.path.join(root, filename)
                print(f"[INFO] Reading: {file_path}")
                df = pd.read_csv(file_path, index_col=False, low_memory=False)

                # Use existing Participant_ID if available
                if 'Participant_ID' in df.columns:
                    df['Participant_ID'] = pd.to_numeric(df['Participant_ID'], errors='coerce')
                else:
                    # Fallback: extract from folder name
                    participant_id = re.search(r'\d+', os.path.basename(os.path.dirname(root)))
                    df['Participant_ID'] = int(participant_id.group(0)) if participant_id else -1

                # Ensure consistency and fix any missing or mixed values
                num_missing_ids = df['Participant_ID'].isna().sum()
                unique_ids = df['Participant_ID'].dropna().unique()

                if num_missing_ids > 0:
                    print(f"[WARNING] {num_missing_ids} missing Participant_IDs in {file_path}")

                if len(unique_ids) == 0:
                    print(f"[WARNING] No valid Participant_IDs found in {file_path}. Setting all to -1.")
                    df['Participant_ID'] = -1
                else:
                    most_common_id = df['Participant_ID'].mode().iloc[0]
                    if len(unique_ids) > 1:
                        print(f"[WARNING] Multiple Participant_IDs found in {file_path}. Replacing all with most common: {most_common_id}")
                    if num_missing_ids > 0 or len(unique_ids) > 1:
                        df['Participant_ID'] = most_common_id
                        print(f"[INFO] All Participant_IDs set to {most_common_id} ({file_path})")

                df.replace('UCDS', -1, inplace=True)

                if "Polar_HearRate_BPM" in df.columns:
                    df.rename(columns={"Polar_HearRate_BPM": "Polar_HeartRate_BPM"}, inplace=True)

                df['Unity_Timestamp'] = pd.to_datetime(
                    df['Unity_Timestamp'], errors="coerce", format="%Y-%m-%dT%H:%M:%S.%fZ"
                ).fillna(
                    pd.to_datetime(df['Unity_Timestamp'], errors="coerce", format="%d-%m-%Y - %H:%M:%S.%f")
                )

                start_time = df['Unity_Timestamp'].iloc[0]
                df['Time_From_Start_Seconds'] = (df['Unity_Timestamp'] - start_time).dt.total_seconds()

                df_list.append(df)

    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()


# --------------------------------------------------------------------------
# NEW: Cached loaders
# --------------------------------------------------------------------------

def load_phys_data_raw():
    pickle_path = os.path.join(config["paths"]["physio"], "phys_data_raw.pkl")
    if os.path.exists(pickle_path):
        print(f"[INFO] Loading phys_data_raw from pickle: {pickle_path}")
        return pd.read_pickle(pickle_path)
    else:
        print("[INFO] No pickle found for phys_data_raw, loading from CSVs...")
        df = load_raw_data(data_path, '_RAW_DATA_')
        df.to_pickle(pickle_path)
        return df

# --------------------------------------------------------------------------
# Uncomment if you still need to fix headers or trailing commas
# fix_headers_and_commas()
# clean_all_csv_trailing_commas(data_path)
# --------------------------------------------------------------------------

# Finally, load DataFrames (from pickles if possible)
#calibration_data_raw = load_calibration_data_raw() 
#calibration_blink_data = load_calibration_blink_data()
phys_data_raw = load_phys_data_raw()

# Load EEG data
eeg_csv_path = os.path.join(config["paths"]["output"], "aggregated", "eeg_features.csv")
eeg_only_data = pd.read_csv(eeg_csv_path)

# Convert to datetime using the raw seconds
eeg_only_data['Window_Start_Second_dt'] = pd.to_datetime('1970-01-01') + \
    pd.to_timedelta(eeg_only_data['Window_Start_Second'], unit='s')



[INFO] Loading phys_data_raw from pickle: E:/PhD_Projects/TSST_Stress_Workload_Pipeline/data/physio\phys_data_raw.pkl


# Resample GSR Data to 10Hz

In [13]:

##############################################
# Setup
##############################################

column_names = [
    'LSL_Timestamp',
    'Pupil_Dilation_Left', 'Pupil_Dilation_Right', 'Current_Blink_Duration',
    'Inter_Blink_Interval', 'Shimmer_D36A_GSR_Skin_Conductance_uS',
    'Shimmer_D36A_GSR_Skin_Resistance_kOhms', 'Polar_HeartRate_BPM',
    'Polar_HeartRate_RR_Interval', 'Foveal_Corrected_Dilation_Left',
    'Foveal_Corrected_Dilation_Right', 'Participant_ID', 'Shown_Scene',
    'Arithmetic_Task', 'Study_Phase', 'Time_From_Start_Seconds', 'Response', 'Response_Time', 'Headset-pX', 'Headset-pY', 'Headset-pZ', 'Headset-TimeStamp'
]

gsr_cols = [
    'Shimmer_D36A_GSR_Skin_Conductance_uS',
    'Shimmer_D36A_GSR_Skin_Resistance_kOhms'
]

timestamp_col = 'LSL_Timestamp'
target_fs = 10
target_interval = pd.Timedelta('100ms')

print("[DEBUG] Columns in phys_data_raw before filtering:")
print(phys_data_raw.columns.tolist())

# Initial cleanup
phys_data_raw.dropna(subset=['LSL_Timestamp', 'Participant_ID'], inplace=True)

# ✅ Only select columns that are actually present
missing_cols = [col for col in column_names if col not in phys_data_raw.columns]
if missing_cols:
    raise KeyError(f"Missing expected column(s): {missing_cols}")

phys_data_raw = phys_data_raw[column_names].copy()

# ✅ Ensure types
phys_data_raw[timestamp_col] = pd.to_datetime(phys_data_raw[timestamp_col], unit='s', errors='coerce')
phys_data_raw['Participant_ID'] = pd.to_numeric(phys_data_raw['Participant_ID'], errors='coerce').astype('Int64')


##############################################
# Downsample GSR only, merge back into other data
##############################################


def resample_gsr_only(df, gsr_cols, target_fs, timestamp_col="LSL_Timestamp"):
    # 1) Sort & dedupe, set timestamp index
    df = df.sort_values(timestamp_col).drop_duplicates(subset=timestamp_col)
    df = df.set_index(timestamp_col)
    
    # 2) Build regular grid at target_fs
    new_index = pd.date_range(
        start=df.index.min(),
        end=df.index.max(),
        freq=pd.Timedelta(seconds=1/target_fs)
    )
    
    # 3) Nearest‐neighbor reindex for GSR columns
    df_nn = df[gsr_cols].reindex(
        new_index,
        method='nearest',
        tolerance=pd.Timedelta('50ms')   # only snap if within 50 ms
    )
    # 4) Fill any remaining gaps
    df_nn = df_nn.ffill().bfill()
    
    # 5) Reset index back to timestamp column
    df_gsr_resampled = df_nn.reset_index().rename(columns={'index': timestamp_col})
    
    # 6) Prepare metadata for merge_asof
    metadata_cols = ['Study_Phase', 'Shown_Scene', 'Arithmetic_Task', 'Time_From_Start_Seconds']
    df_meta = df[metadata_cols].reset_index()
    df_meta[timestamp_col] = df_meta[timestamp_col].dt.round('1ms')
    df_gsr_resampled[timestamp_col] = df_gsr_resampled[timestamp_col].dt.round('1ms')
    
    # 7) Merge metadata back on nearest timestamp
    df_final = pd.merge_asof(
        df_gsr_resampled.sort_values(timestamp_col),
        df_meta.sort_values(timestamp_col),
        on=timestamp_col,
        direction='nearest',
        tolerance=pd.Timedelta("200ms")
    )
    
    # 8) Add participant + sample rate columns if needed
    # (Assuming 'Participant_ID' and you want to tag sample rate)
    if 'Participant_ID' in df.columns:
        df_final['Participant_ID'] = df['Participant_ID'].iloc[0]
    df_final['Sample_Rate_Hz'] = target_fs
    
    return df_final

# Usage:
resampled_gsr_parts = []
for pid, sub in phys_data_raw.groupby("Participant_ID"):
    gsr_res = resample_gsr_only(sub, gsr_cols, target_fs, timestamp_col)
    if gsr_res is not None:
        resampled_gsr_parts.append(gsr_res)
gsr_only_resampled_data = pd.concat(resampled_gsr_parts).reset_index(drop=True)



# Compute actual sampling intervals
intervals = gsr_only_resampled_data.groupby('Participant_ID')[timestamp_col].apply(lambda x: x.diff().dt.total_seconds().dropna())

# Convert to Hz and compare to expected
actual_fs = (1 / intervals.groupby('Participant_ID').median()).round(2)
print("\n[CHECK] Actual sampling rate per participant (Hz):")
print(actual_fs)

# Optional: Compare to stated Sample_Rate_Hz
print("\n[INFO] Unique 'Sample_Rate_Hz' values in final df:")
print(gsr_only_resampled_data['Sample_Rate_Hz'].unique())




[DEBUG] Columns in phys_data_raw before filtering:
['Frame_Count', 'Unity_Timestamp', 'LSL_Timestamp', 'Participant_ID', 'Participant_Age', 'Participant_Calibration_Resting_HR', 'Participant_Calibration_GSR_Conductance', 'Participant_Calibration_GSR_Resistance', 'Unity Scene', 'Study_Phase', 'Shown_Scene', 'Arithmetic_Task', 'Participant_State', 'Exposure_Scene_Timer', 'Task_State', 'Feedback_State', 'Response_Time', 'Waiting_Room_State', 'Response', 'Headset-TimeStamp', 'Headset-pX', 'Headset-pY', 'Headset-pZ', 'Headset-rX', 'Headset-rY', 'Headset-rZ', 'Headset-arX', 'Headset-arY', 'Headset-arZ', 'Headset-aqX', 'Headset-aqY', 'Headset-aqZ', 'Headset-aqW', 'abstract-TimeStamp', 'abstract-Mark', 'EEG-TimeStamp', 'EEG-EE225-000000-000495-02-eego_laptop-0', 'EEG-EE225-000000-000495-02-eego_laptop-1', 'EEG-EE225-000000-000495-02-eego_laptop-2', 'EEG-EE225-000000-000495-02-eego_laptop-3', 'EEG-EE225-000000-000495-02-eego_laptop-4', 'EEG-EE225-000000-000495-02-eego_laptop-5', 'EEG-EE225-0000

KeyboardInterrupt: 

In [ ]:
# ========================
# Cleaning Functions
# ========================

def lowpass_filter(input_signal, cutoff, fs):
    """
    Apply a standardised low-pass filter using NeuroKit2's built-in method.

    Parameters:
        input_signal (array-like): Signal to filter.
        cutoff (float): Cutoff frequency in Hz.
        fs (float): Sampling rate in Hz.

    Returns:
        np.ndarray: Filtered signal.
    """
    return nk.signal_filter(input_signal, sampling_rate=fs, highcut=cutoff, lowcut=None, method="butterworth")



def clean_eda_signal(raw_eda, sampling_rate=10):
    """
    Clean GSR/EDA signal using NeuroKit2's standardised method.

    Parameters:
        raw_eda (array-like): Raw EDA signal.
        sampling_rate (int): Sampling rate in Hz.

    Returns:
        np.ndarray: Cleaned EDA signal.
    """
    return nk.eda_clean(raw_eda, sampling_rate=sampling_rate)

def clean_gsr_data(df, columns_to_clean, fs=30):
    """
    Clean GSR/EDA signals using NeuroKit2's built-in eda_clean method.

    Parameters:
        df (DataFrame): DataFrame containing GSR columns.
        columns_to_clean (list): List of GSR column names.
        fs (int): Sampling frequency in Hz.

    Returns:
        DataFrame: Updated DataFrame with cleaned GSR columns.
    """
    print("Cleaning GSR using neurokit2.eda_clean...")
    
    for col in columns_to_clean:
        if col in df.columns:
            cleaned_col = f"{col}_CLEANED_NK"
            try:
                df[cleaned_col] = nk.eda_clean(df[col], sampling_rate=fs)
            except Exception as e:
                logging.warning(f"[GSR] Failed to clean {col} with nk.eda_clean(): {e}")
                df[cleaned_col] = np.nan
        else:
            logging.warning(f"[GSR] Column '{col}' not found in DataFrame.")
    
    print("GSR cleaning complete.")
    return df
# ------------------------
# Additional Cleaning Steps
# ------------------------


def apply_absolute_threshold_cleaning(df, col, min_val, max_val, logger=None, participant_id=None, condition_name=None):
    cleaned_col = f"{col}_CLEANED_ABS"
    
    total = df[col].shape[0]
    n_out_of_bounds = (~df[col].between(min_val, max_val)).sum()
    
    df[cleaned_col] = df[col].where((df[col] >= min_val) & (df[col] <= max_val))

    if logger:
        prefix = ""
        if participant_id:
            prefix += f"[{participant_id}] "
        if condition_name:
            prefix += f"[{condition_name}] "
        
        logger.info(f"{prefix}{col}: {n_out_of_bounds} / {total} values set to NaN (outside [{min_val}, {max_val}])")

    print(f"Absolute threshold cleaning applied for column '{col}'. New column '{cleaned_col}' created.")
    return df


def plot_raw_vs_cleaned(raw_signal, cleaned_signal, timestamps=None,
                        title="Raw vs Cleaned Signal", ylabel="Signal Value"):
    """
    Plot raw vs cleaned signals.

    Parameters:
        raw_signal (array-like): Original raw signal.
        cleaned_signal (array-like): Processed (cleaned) signal.
        timestamps (array-like, optional): Time data for x-axis.
        title (str): Plot title.
        ylabel (str): Label for y-axis.

    Returns:
        None
    """
    plt.figure(figsize=(12, 6))
    if timestamps is not None:
        plt.plot(timestamps, raw_signal, label="Raw Signal", alpha=0.6)
        plt.plot(timestamps, cleaned_signal, label="Cleaned Signal", alpha=0.8)
    else:
        plt.plot(raw_signal, label="Raw Signal", alpha=0.6)
        plt.plot(cleaned_signal, label="Cleaned Signal", alpha=0.8)

    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Clean HR Data

In [ ]:

# --------------------------------------------------------------
# Set up
# --------------------------------------------------------------
# assume phys_data_raw, qc_loggers dict already exist
bpm_thresholds = {"Polar_HeartRate_BPM": (40, 220)}
rr_raw_col     = "Polar_HeartRate_RR_Interval"
rr_clean_col   = "Polar_HeartRate_RR_Interval_CLEANED_ABS"
rr_thresholds  = (100, 2000)  # valid RR range (ms)

all_cleaned_parts = []

# --------------------------------------------------------------
# Main HR Cleaning + QC‐logging Loop
# --------------------------------------------------------------
for (pid, cond), df_cond in phys_data_raw.groupby(["Participant_ID", "Study_Phase"]):
    participant_id  = int(pid)
    condition_name  = cond
    logger          = qc_loggers.get(participant_id)
    if logger is None:
        print(f"⚠️  No QC logger for P{participant_id}, skipping.")
        continue

    logger.info(f"Logger retrieved for P{participant_id}, condition='{condition_name}'")
    logger.info(f"[{participant_id}] [{condition_name}] Starting HR cleaning pipeline")

    # Make a copy for safety
    df = df_cond.copy()

    # 0) Bootstrap cleaned RR if missing
    if rr_clean_col not in df and rr_raw_col in df:
        df[rr_clean_col] = df[rr_raw_col].copy()

    # 1) Clamp extremes on RR
    if rr_clean_col in df:
        df.loc[~df[rr_clean_col].between(*rr_thresholds), rr_clean_col] = np.nan

    # 2) BPM thresholding
    for col, (vmin, vmax) in bpm_thresholds.items():
        if col in df:
            df = apply_absolute_threshold_cleaning(
                df, col, vmin, vmax,
                logger=logger,
                participant_id=participant_id,
                condition_name=condition_name
            )

    # 3) Interpolation
    bpm_col = "Polar_HeartRate_BPM_CLEANED_ABS"
    if bpm_col in df:
        df[bpm_col] = df[bpm_col].interpolate(method="linear", limit=10, limit_direction="both")

    # 4) Ectopic beat correction + MAD filtering
    if rr_clean_col in df:
        rr_ms = df[rr_clean_col].dropna().values
        sr = 1000
        peaks = nk.intervals_to_peaks(rr_ms, sampling_rate=sr)
        _, peaks_clean = nk.signal_fixpeaks(peaks, sampling_rate=sr, method="Kubios")
        diffs = np.diff(peaks_clean)
        rr_clean_ms = diffs * (1000.0 / sr)

        orig_idx = df.index[df[rr_clean_col].notna()].to_numpy()
        n_assign = min(len(orig_idx) - 1, len(rr_clean_ms))
        assign_idx = orig_idx[1 : 1 + n_assign]
        df.loc[assign_idx, rr_clean_col] = rr_clean_ms[:n_assign]

        # MAD-based outlier rejection
        series = df[rr_clean_col]
        mad = series.rolling(11, center=True).apply(
            lambda x: np.median(np.abs(x - np.median(x))), raw=True
        )
        floor = mad[mad > 0].min() if mad[mad > 0].any() else 10
        mad = mad.fillna(floor).clip(lower=floor)
        med = series.rolling(11, center=True).median()
        outliers = np.abs(series - med) > (4 * mad)
        df.loc[outliers, rr_clean_col] = np.nan

        # interpolate small gaps
        df[rr_clean_col] = df[rr_clean_col].interpolate(method="cubic", limit=2, limit_direction="both")

    # Final logging
    cleaned_cols = [c for c in df.columns if c.endswith("_CLEANED_ABS")]
    total = len(df)
    for col in cleaned_cols:
        retained = df[col].notna().sum()
        logger.info(f"[{participant_id}] [{condition_name}] {col}: {retained} / {total}")

    # Store
    all_cleaned_parts.append(df)

# --------------------------------------------------------------
# Reassemble full cleaned DataFrame
# --------------------------------------------------------------
cleaned_phys_data = pd.concat(all_cleaned_parts, ignore_index=True)
logging.info("✅ All participants processed. Cleaned data ready.")



Absolute threshold cleaning applied for column 'Polar_HeartRate_BPM'. New column 'Polar_HeartRate_BPM_CLEANED_ABS' created.
[1] [Calibrations] rr_ms: n=8638, mean=924.3896735355406, sd=83.09775509561572
[1] [Calibrations] RMSSD pre=7.953 ms, post=7.632 ms, step=fixpeaks
[1] [Calibrations] MAD outliers: 4 (0.0% of valid)
[1] [Calibrations] RR cubic interp filled=4
Absolute threshold cleaning applied for column 'Polar_HeartRate_BPM'. New column 'Polar_HeartRate_BPM_CLEANED_ABS' created.
[1] [End Blank Fixation] rr_ms: n=2745, mean=955.624043715847, sd=36.77214839126977
[1] [End Blank Fixation] RMSSD pre=5.113 ms, post=5.089 ms, step=fixpeaks
[1] [End Blank Fixation] MAD outliers: 0 (0.0% of valid)
[1] [End Blank Fixation] RR cubic interp filled=0
Absolute threshold cleaning applied for column 'Polar_HeartRate_BPM'. New column 'Polar_HeartRate_BPM_CLEANED_ABS' created.
[1] [End Room Fixation] rr_ms: n=3144, mean=906.5340330788804, sd=70.47813883003346
[1] [End Room Fixation] RMSSD pre=6.9

KeyboardInterrupt: 

In [17]:
import pandas as pd, numpy as np

def rmssd(x):
    x = np.diff(x)
    return np.sqrt(np.mean(x**2)) if len(x) > 0 else np.nan

# ----------------------------------------------------------
# Clamp extreme RR intervals before analysis
# ----------------------------------------------------------
rr_col = "Polar_HeartRate_RR_Interval"
lower, upper = 200, 2000  # ms valid range

# Identify and report bad values
bad_mask = ~phys_data_raw[rr_col].between(lower, upper)
n_bad = bad_mask.sum()
print(f"⚠️  Found {n_bad} RR intervals outside {lower}-{upper} ms")

if n_bad > 0:
    print(phys_data_raw.loc[bad_mask, ["Participant_ID", "Study_Phase", rr_col]]
          .groupby("Participant_ID")[rr_col]
          .describe().round(2))

# Clamp invalid values to NaN
phys_data_raw.loc[bad_mask, rr_col] = np.nan

# ----------------------------------------------------------
# Compute per-participant-condition RMSSD summary
# ----------------------------------------------------------
check = []
for (pid, cond), dfc in phys_data_raw.groupby(["Participant_ID", "Study_Phase"]):
    rr = dfc[rr_col].dropna().values
    if len(rr) > 30:
        check.append({
            "pid": pid,
            "cond": cond,
            "mean_rr": np.mean(rr),
            "rmssd": rmssd(rr),
            "n": len(rr)
        })

df_check = pd.DataFrame(check)

print("\n=== RR and RMSSD Summary (after clamping) ===")
print(df_check.describe()[["mean_rr", "rmssd"]].round(3))


⚠️  Found 95225 RR intervals outside 200-2000 ms
                  count      mean       std      min      25%      50%  \
Participant_ID                                                           
1.0                61.0  60991.00      0.00  60991.0  60991.0  60991.0   
2.0               114.0  47147.63      6.55  47135.0  47151.0  47151.0   
3.0               167.0  53574.47      8.01  53567.0  53567.0  53567.0   
4.0              1433.0   2431.74      3.35   2431.0   2431.0   2431.0   
5.0                23.0  57007.00      0.00  57007.0  57007.0  57007.0   
6.0                99.0  37789.37  19501.11   2520.0  48463.0  48463.0   
7.0                71.0  57055.00      0.00  57055.0  57055.0  57055.0   
9.0              1443.0   9519.17    339.54   7839.0   9588.0   9588.0   
10.0             2776.0  47887.60  15426.63   2025.0  53682.0  53682.0   
11.0              116.0  61295.00      0.00  61295.0  61295.0  61295.0   
12.0               47.0  21711.00      0.00  21711.0  21711.0  

# Cell 6.1: Rolling Aggregation: Data Cleaning and Definitions

# Clean Eye Data

In [ ]:

# =============================================================================
# 0) Configuration & snapshot
# =============================================================================
raw_eye_data = cleaned_phys_data.copy()  # start from your physio‐cleaned data

thresholds = {
    "Inter_Blink_Interval":            (0.1, 20.0),   # s
    "Current_Blink_Duration":          (0.05, 1.0),   # s
    "Foveal_Corrected_Dilation_Left":  (-4.0, 4.0),   # mm
    "Foveal_Corrected_Dilation_Right": (-4.0, 4.0),   # mm
}

blink_params = {
    "low_margin":          0.5,   # mm above abs threshold
    "max_interp_duration": 0.3,   # s
}

TIME_COL = "LSL_Timestamp"

global_acc = {}

# ensure datetime
if not pd.api.types.is_datetime64_any_dtype(raw_eye_data[TIME_COL]):
    raw_eye_data[TIME_COL] = pd.to_datetime(raw_eye_data[TIME_COL])

all_cleaned_parts = []

# =============================================================================
# 1) Per‐participant loop
# =============================================================================
for pid, df_pid in raw_eye_data.groupby("Participant_ID", sort=False):
    participant_id = int(pid)
    condition_name = df_pid['Study_Phase'].iloc[0]
    logger = qc_loggers.get(participant_id)
    if logger is None:
        print(f"[⚠️] No QC logger for P{participant_id}, skipping eye‐cleaning.")
        continue

    logger.info(f"[{participant_id}] [{condition_name}] Starting eye‐feature cleaning")
    df = df_pid.sort_values(TIME_COL).set_index(TIME_COL).copy()

    # accumulators for this participant
    acc = {f"{feat}_CLEANED_ABS": {"dropped": 0, "retained": 0, "total": 0}
           for feat in thresholds}

    # --- 1) Absolute‐threshold cleaning ---
    for feat, (mn, mx) in thresholds.items():
        clean_col = f"{feat}_CLEANED_ABS"
        before = df[feat].notna().sum()
        df[feat] = pd.to_numeric(df[feat], errors="coerce")
        df[clean_col] = df[feat].where(df[feat].between(mn, mx), other=np.nan)
        after = df[clean_col].notna().sum()
        total = len(df)
        global_acc.setdefault(clean_col, {"dropped": 0, "retained": 0, "total": 0})
        global_acc[clean_col]["dropped"]  += before - after
        global_acc[clean_col]["retained"] += after
        global_acc[clean_col]["total"]    += total
        logger.info(
            f"[{participant_id}] [{condition_name}] "
            f"{clean_col}: retained {after}/{total} (dropped {before-after})"
        )

    # --- 2) Closure detection & classification ---
    Lcol = "Foveal_Corrected_Dilation_Left_CLEANED_ABS"
    Rcol = "Foveal_Corrected_Dilation_Right_CLEANED_ABS"
    mnL, _ = thresholds["Foveal_Corrected_Dilation_Left"]
    mnR, _ = thresholds["Foveal_Corrected_Dilation_Right"]
    lowL = mnL + blink_params["low_margin"]
    lowR = mnR + blink_params["low_margin"]
    drop_thr = blink_params.get("drop_thresh", 1.0)

    dt = df.index.to_series().diff().dt.total_seconds()
    rate_L = df[Lcol].diff() / dt
    rate_R = df[Rcol].diff() / dt

    is_low_L  = df[Lcol] <= lowL
    is_low_R  = df[Rcol] <= lowR
    is_drop_L = rate_L <= -drop_thr
    is_drop_R = rate_R <= -drop_thr
    is_closure = (is_low_L & is_low_R) & (is_drop_L | is_drop_R)

    grp = (is_closure != is_closure.shift(fill_value=False)).cumsum()
    durations = (
        is_closure
        .groupby(grp)
        .apply(lambda seg: (seg.index[-1] - seg.index[0]).total_seconds())
    )

    short_ids     = durations[durations <= blink_params["max_interp_duration"]].index
    prolonged_ids = durations[durations >= blink_params.get("min_remove_duration", np.inf)].index
    medium_ids    = durations.index.difference(short_ids.union(prolonged_ids))

    logger.info(
        f"[{participant_id}] [{condition_name}] "
        f"Closures detected → short: {len(short_ids)}, medium: {len(medium_ids)}, prolonged: {len(prolonged_ids)}"
    )

    # mask all closures (interpolate short ones, leave others as NaN)
    df.loc[grp.isin(short_ids)   & is_closure, [Lcol, Rcol]] = np.nan
    df.loc[grp.isin(medium_ids)  & is_closure, [Lcol, Rcol]] = np.nan
    df.loc[grp.isin(prolonged_ids)& is_closure, [Lcol, Rcol]] = np.nan

    # --- 3) Interpolation on all *_CLEANED_ABS columns ---
    dt_med = dt.median()
    max_n  = int(blink_params["max_interp_duration"] // dt_med) if dt_med>0 else None
    clean_cols = [f"{feat}_CLEANED_ABS" for feat in thresholds]

    before_interp = df[clean_cols].isna().sum().sum()
    df[clean_cols] = df[clean_cols].interpolate(
        method='time', limit=max_n, limit_direction='both'
    )
    after_interp = df[clean_cols].isna().sum().sum()
    interpolated = before_interp - after_interp
    logger.info(
        f"[{participant_id}] [{condition_name}] "
        f"Interpolated {interpolated} NaNs across {clean_cols}"
    )

    all_cleaned_parts.append(df.reset_index())

# =============================================================================
# 2) Reassemble & per‐condition summary logging
# =============================================================================
eye_cleaned = pd.concat(all_cleaned_parts, ignore_index=True)

for pid, df_pid in eye_cleaned.groupby("Participant_ID", sort=False):
    participant_id = int(pid)
    logger = qc_loggers.get(participant_id)
    if logger is None: 
        continue
    for cond, df_cond in df_pid.groupby("Study_Phase", sort=False):
        for col in [f"{feat}_CLEANED_ABS" for feat in thresholds]:
            if col in df_cond:
                retained = df_cond[col].notna().sum()
                total    = len(df_cond)
                logger.info(
                    f"[{participant_id}] [{cond}] {col}: {retained} / {total} values retained after all cleaning"
                )
            else:
                logger.warning(
                    f"[{participant_id}] [{cond}] Column '{col}' not found in eye‐cleaned data"
                )

# finally, replace your cleaned_phys_data with the eye-enhanced version:
cleaned_phys_data = eye_cleaned.copy()



# =============================================================================
# DIAGNOSTICS & SANITY CHECKS
# =============================================================================


# B) Overall dropped/retained summary
print("\n=== Overall Dropped/Retained Summary ===")
for clean_col, stats in global_acc.items():
    total = stats["total"]
    drp   = stats["dropped"]
    keep  = stats["retained"]

    if total == 0:
        # no data for this metric at all
        print(f"{clean_col}: no observations to summarize.")
        continue

    print(
        f"{clean_col}: dropped {drp}/{total} ({drp/total:.1%}), "
        f"kept    {keep}/{total} ({keep/total:.1%})"
    )




--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\u2010' in position 57: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\User


=== Overall Dropped/Retained Summary ===
Inter_Blink_Interval_CLEANED_ABS: dropped 1104271/5080562 (21.7%), kept    3976291/5080562 (78.3%)
Current_Blink_Duration_CLEANED_ABS: dropped 1578382/5080562 (31.1%), kept    3502180/5080562 (68.9%)
Foveal_Corrected_Dilation_Left_CLEANED_ABS: dropped 448335/5080562 (8.8%), kept    4632227/5080562 (91.2%)
Foveal_Corrected_Dilation_Right_CLEANED_ABS: dropped 434506/5080562 (8.6%), kept    4646056/5080562 (91.4%)


In [ ]:
all_cleaned_gsr_parts = []

gsr_cleaned_cols = [
    'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS',
]

def remove_flat_signals(df, col, max_flat_duration_sec, fs, logger=None):
    max_flat_samples = int(max_flat_duration_sec * fs)
    
    if col not in df.columns:
        print(f"[DEBUG] Column '{col}' not found in DataFrame.")
        return df

    print(f"[DEBUG] Checking for flatlines in '{col}' (>{max_flat_samples} samples)")

    values = df[col].values
    flat_mask = np.zeros_like(values, dtype=bool)
    total_flagged = 0
    run_counter = 0

    start_idx = 0
    while start_idx < len(values):
        current_val = values[start_idx]
        run_len = 1
        while (
            start_idx + run_len < len(values)
            and values[start_idx + run_len] == current_val
            and not np.isnan(current_val)
        ):
            run_len += 1

        if run_len >= max_flat_samples:
            flat_mask[start_idx:start_idx + run_len] = True
            total_flagged += run_len
            run_counter += 1
            print(f"[DEBUG] Flatline detected: {run_len} samples at index {start_idx} (value = {current_val})")

        start_idx += run_len

    df.loc[flat_mask, col] = np.nan

    if logger:
        logger.info(f"[GSR] {col}: Removed {total_flagged} samples across {run_counter} flat segments > {max_flat_duration_sec}s")

    print(f"[DEBUG] Total {total_flagged} flat samples removed in '{col}' across {run_counter} segments.")

    return df



for pid, df_gsr in gsr_only_resampled_data.groupby("Participant_ID"):
    participant_id = int(pid)
    logger = qc_loggers.get(participant_id)
    
    if logger is None:
        print(f"[GSR] Logger missing for P{participant_id}, skipping.")
        continue

    print(f"[GSR] Cleaning resampled GSR for P{participant_id}")

    # Apply absolute threshold cleaning
    for col, (min_val, max_val) in {
        "Shimmer_D36A_GSR_Skin_Conductance_uS": (0.01, 30),
    }.items():
        if col in df_gsr.columns:
            df_gsr = apply_absolute_threshold_cleaning(
                df_gsr, col, min_val, max_val,
                logger=logger,
                participant_id=participant_id,
                condition_name="GSR_Resampled"
            )
        else:
            logger.warning(f"[GSR] Column '{col}' not found for P{participant_id}")

    # Clean GSR signal
    df_gsr = clean_gsr_data(
        df_gsr,
        columns_to_clean=[
            'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS',
        ],
        fs=target_fs
    )

    # Log retention BEFORE interpolation
    for condition_name, df_cond in df_gsr.groupby("Study_Phase"):
        for col in gsr_cleaned_cols:
            if col in df_cond.columns:
                non_nan = df_cond[col].notna().sum()
                total = len(df_cond[col])
                logger.info(f"[{participant_id}] [{condition_name}] {col}: {non_nan} / {total} values retained BEFORE interpolation")
            else:
                logger.warning(f"[{participant_id}] [{condition_name}] {col}: Column not found in GSR data")


    # Remove flat signals before interpolation
    for col in [
        'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS',
    ]:
        df_gsr = remove_flat_signals(df_gsr, col, max_flat_duration_sec=5, fs=target_fs, logger=logger)

    # Interpolate only small gaps (≤ 5 seconds)
    interpolation_limit = int(5 * target_fs)
    for col in [
        'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS',
    ]:
        if col in df_gsr.columns:
            before = df_gsr[col].isna().sum()
            df_gsr[col] = df_gsr[col].interpolate(method='linear',
                                                limit=interpolation_limit,
                                                limit_direction='both')
            after = df_gsr[col].isna().sum()
            logger.info(f"[GSR] P{participant_id} interpolated {before - after} values in '{col}'")
            logger.info(f"[GSR] P{participant_id} remaining NaNs in '{col}': {after}")
        else:
            logger.warning(f"[GSR] Column '{col}' missing in P{participant_id}")

    all_cleaned_gsr_parts.append(df_gsr)

cleaned_gsr_data = pd.concat(all_cleaned_gsr_parts).reset_index(drop=True)


for df_pid in all_cleaned_gsr_parts:
    participant_id = int(df_pid['Participant_ID'].iloc[0])
    logger = qc_loggers.get(participant_id)

    if logger is None:
        print(f"[QC] Logger not found for P{participant_id}, skipping GSR summary logging.")
        continue

    print(f"[QC] Summarising cleaned GSR for P{participant_id}")

    if "Study_Phase" not in df_pid.columns:
        logger.warning(f"[{participant_id}] No 'Study_Phase' column found in cleaned data.")
        continue

    for condition_name, df_cond in df_pid.groupby("Study_Phase"):
        for col in gsr_cleaned_cols:
            if col in df_cond.columns:
                total = len(df_cond[col])
                non_nan = df_cond[col].notna().sum()
                num_nan = total - non_nan
                percent_retained = 100 * non_nan / total if total > 0 else 0
                logger.info(
                    f"[{participant_id}] [{condition_name}] {col}: "
                    f"{non_nan} / {total} retained "
                    f"({percent_retained:.1f}%), {num_nan} missing"
                )
            else:
                logger.warning(f"[{participant_id}] [{condition_name}] {col}: Column not found in GSR data")




[GSR] Cleaning resampled GSR for P1
Absolute threshold cleaning applied for column 'Shimmer_D36A_GSR_Skin_Conductance_uS'. New column 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' created.
Cleaning GSR using neurokit2.eda_clean...
GSR cleaning complete.
[DEBUG] Checking for flatlines in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' (>50 samples)
[DEBUG] Flatline detected: 435 samples at index 101 (value = 2.02857142857143)
[DEBUG] Flatline detected: 88 samples at index 3755 (value = 1.4043956043956)
[DEBUG] Flatline detected: 686 samples at index 4611 (value = 1.31668331668332)
[DEBUG] Flatline detected: 542 samples at index 5369 (value = 1.87619047619048)
[DEBUG] Flatline detected: 964 samples at index 19438 (value = 4.28351648351648)
[DEBUG] Flatline detected: 103 samples at index 21291 (value = 2.003663003663)
[DEBUG] Total 2818 flat samples removed in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' across 6 segments.
[GSR] Cleaning resampled GSR for P2
Absolute thresho

C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:69: NeuroKitWarning: There are 3 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:94: FutureWarning: DataFrame.pad/Series.pad is deprecated. Use DataFrame.ffill/Series.ffill instead
  eda_signal = pd.DataFrame.pad(pd.Series(eda_signal))


[DEBUG] Total 921 flat samples removed in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' across 2 segments.
[GSR] Cleaning resampled GSR for P7
Absolute threshold cleaning applied for column 'Shimmer_D36A_GSR_Skin_Conductance_uS'. New column 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' created.
Cleaning GSR using neurokit2.eda_clean...
GSR cleaning complete.
[DEBUG] Checking for flatlines in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' (>50 samples)
[DEBUG] Flatline detected: 498 samples at index 63 (value = 10.310015188064)
[DEBUG] Total 498 flat samples removed in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' across 1 segments.
[GSR] Cleaning resampled GSR for P8
Absolute threshold cleaning applied for column 'Shimmer_D36A_GSR_Skin_Conductance_uS'. New column 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' created.
Cleaning GSR using neurokit2.eda_clean...
GSR cleaning complete.
[DEBUG] Checking for flatlines in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_AB

C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:69: NeuroKitWarning: There are 549 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:94: FutureWarning: DataFrame.pad/Series.pad is deprecated. Use DataFrame.ffill/Series.ffill instead
  eda_signal = pd.DataFrame.pad(pd.Series(eda_signal))


GSR cleaning complete.
[DEBUG] Checking for flatlines in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' (>50 samples)
[DEBUG] Flatline detected: 392 samples at index 637 (value = 2.61465201465201)
[DEBUG] Flatline detected: 51 samples at index 16881 (value = 2.92380952380952)
[DEBUG] Flatline detected: 484 samples at index 26255 (value = 3.35164835164835)
[DEBUG] Flatline detected: 281 samples at index 26795 (value = 3.04395604395604)
[DEBUG] Flatline detected: 818 samples at index 27169 (value = 2.73333333333333)
[DEBUG] Total 2026 flat samples removed in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' across 5 segments.
[GSR] Cleaning resampled GSR for P14
Absolute threshold cleaning applied for column 'Shimmer_D36A_GSR_Skin_Conductance_uS'. New column 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' created.
Cleaning GSR using neurokit2.eda_clean...
GSR cleaning complete.
[DEBUG] Checking for flatlines in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' (>50 samples)
[DEB

C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:69: NeuroKitWarning: There are 484 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:94: FutureWarning: DataFrame.pad/Series.pad is deprecated. Use DataFrame.ffill/Series.ffill instead
  eda_signal = pd.DataFrame.pad(pd.Series(eda_signal))
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:69: NeuroKitWarning: There are 1 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5

[DEBUG] Total 522 flat samples removed in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' across 1 segments.
[GSR] Cleaning resampled GSR for P26
Absolute threshold cleaning applied for column 'Shimmer_D36A_GSR_Skin_Conductance_uS'. New column 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' created.
Cleaning GSR using neurokit2.eda_clean...
GSR cleaning complete.
[DEBUG] Checking for flatlines in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' (>50 samples)
[DEBUG] Flatline detected: 756 samples at index 584 (value = 2.09450549450549)
[DEBUG] Flatline detected: 71 samples at index 7026 (value = 1.95677655677656)
[DEBUG] Total 827 flat samples removed in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' across 2 segments.
[GSR] Cleaning resampled GSR for P27
Absolute threshold cleaning applied for column 'Shimmer_D36A_GSR_Skin_Conductance_uS'. New column 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' created.
Cleaning GSR using neurokit2.eda_clean...
GSR cleaning complete.

C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:69: NeuroKitWarning: There are 51 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:94: FutureWarning: DataFrame.pad/Series.pad is deprecated. Use DataFrame.ffill/Series.ffill instead
  eda_signal = pd.DataFrame.pad(pd.Series(eda_signal))
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\eda\eda_clean.py:69: NeuroKitWarning: There are 118 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz

[DEBUG] Total 653 flat samples removed in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' across 4 segments.
[GSR] Cleaning resampled GSR for P40
Absolute threshold cleaning applied for column 'Shimmer_D36A_GSR_Skin_Conductance_uS'. New column 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' created.
Cleaning GSR using neurokit2.eda_clean...
GSR cleaning complete.
[DEBUG] Checking for flatlines in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' (>50 samples)
[DEBUG] Flatline detected: 395 samples at index 34 (value = 0.747474747474747)
[DEBUG] Flatline detected: 64 samples at index 6273 (value = 0.407370407370407)
[DEBUG] Total 459 flat samples removed in 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' across 2 segments.
[GSR] Cleaning resampled GSR for P41
Absolute threshold cleaning applied for column 'Shimmer_D36A_GSR_Skin_Conductance_uS'. New column 'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS' created.
Cleaning GSR using neurokit2.eda_clean...
GSR cleaning complete

In [ ]:
"""
This cell computes two sets of statistics from the physiological data:
  1. Rolling window statistics (short-term, dynamic measures)
  2. Full condition statistics (longer-term, baseline/aggregate measures)

For every metric listed in ALL_PHYS_COLUMNS (and baseline measures in BASELINE_MEASURES),
both a rolling version (using a ~15-second window) and a full condition version (using the entire
condition duration from EEG data) are computed.
"""

# List of all physiological columns to include
HR_COLUMNS = [
    'Polar_HeartRate_BPM_CLEANED_ABS',
    'Polar_HeartRate_RR_Interval_CLEANED_ABS',
    'Polar_HeartRate_BPM',
    'Polar_HeartRate_RR_Interval',
]

GSR_COLUMNS = [
    'Shimmer_D36A_GSR_Skin_Conductance_uS',
    'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS',
    'Shimmer_D36A_GSR_Skin_Conductance_uS_CLEANED_ABS_CLEANED_NK'
]

OTHER_COLUMNS = [
    'Pupil_Dilation_Left',
    'Pupil_Dilation_Right',
    'Foveal_Corrected_Dilation_Left',
    'Foveal_Corrected_Dilation_Right',
    'Current_Blink_Duration',
    'Inter_Blink_Interval',
    'Foveal_Corrected_Dilation_Left_CLEANED_ABS',
    'Foveal_Corrected_Dilation_Right_CLEANED_ABS',
    'Inter_Blink_Interval_CLEANED_ABS',
     'Current_Blink_Duration_CLEANED_ABS',
     "Head_Velocity"
]


ALL_PHYS_COLUMNS = HR_COLUMNS + GSR_COLUMNS + OTHER_COLUMNS


# List of baseline measures (if available)
BASELINE_MEASURES = [
    'Median_0_RestingHR_Baseline',
    'Mean_0_RestingHR_Baseline',
    'SD_0_RestingHR_Baseline',
    'Median_1_RestingHR_Interval_Baseline',
    'Mean_1_RestingHR_Interval_Baseline',
    'SD_1_RestingHR_Interval_Baseline',
    'Median_0_GSRConductance_Baseline',
    'Mean_0_GSRConductance_Baseline',
    'SD_0_GSRConductance_Baseline',
    'Median_1_GSRResistance_Baseline',
    'Mean_1_GSRResistance_Baseline',
    'SD_1_GSRResistance_Baseline'
]



# --- Data Preparation ---

# 1. Convert phys columns in cleaned_phys_data
for col in HR_COLUMNS + OTHER_COLUMNS:  # exclude GSR columns
    if col in cleaned_phys_data.columns:
        cleaned_phys_data[col] = pd.to_numeric(cleaned_phys_data[col], errors='coerce')

# 2. Convert GSR columns in cleaned_gsr_data
for col in GSR_COLUMNS:
    if col in cleaned_gsr_data.columns:
        cleaned_gsr_data[col] = pd.to_numeric(cleaned_gsr_data[col], errors='coerce')

# 3. Participant ID cleanup
for df in [cleaned_phys_data, cleaned_gsr_data]:
    df['Participant_ID'] = pd.to_numeric(df['Participant_ID'], errors='coerce').fillna(-1).astype(int)

# 4. Strip whitespace from column names
eeg_only_data.columns = eeg_only_data.columns.str.strip()
cleaned_phys_data.columns = cleaned_phys_data.columns.str.strip()
cleaned_gsr_data.columns = cleaned_gsr_data.columns.str.strip()

# 5. Rename Participant column if needed
if 'Participant' in eeg_only_data.columns and 'Participant_ID' not in eeg_only_data.columns:
    eeg_only_data.rename(columns={'Participant': 'Participant_ID'}, inplace=True)

# Print out the column names in eeg_only_data and preview
print("=== Columns in eeg_only_data ===")
print(eeg_only_data.columns.tolist())

print("\n=== First few rows of eeg_only_data ===")
print(eeg_only_data.head())


###############################################
# Function: Calculate Rolling Window Stats
###############################################


def calculate_rolling_stats_all(cleaned_phys_data, cleaned_gsr_data, eeg_only_data):
    print("[INFO] Starting rolling statistics calculation with all desired columns...")

    # Ensure time columns are datetime
    if not np.issubdtype(cleaned_phys_data['Time_From_Start_Seconds'].dtype, np.datetime64):
        cleaned_phys_data['Time_From_Start_Seconds'] = pd.to_datetime('1970-01-01') + \
            pd.to_timedelta(cleaned_phys_data['Time_From_Start_Seconds'], unit='s')
    if not np.issubdtype(eeg_only_data['Window_Start_Second'].dtype, np.datetime64):
        eeg_only_data['Window_Start_Second'] = pd.to_datetime('1970-01-01') + \
            pd.to_timedelta(eeg_only_data['Window_Start_Second'], unit='s')

    results = []


    for idx, row in tqdm(eeg_only_data.iterrows(), total=len(eeg_only_data), desc="Calculating Rolling Stats"):
        try:
            participant = row['Participant_ID']
            sample_frame = row['Sample_Frame']
            window_start_second = row['Window_Start_Second']

            window_start = window_start_second
            window_end = window_start_second + pd.Timedelta(seconds=30)

            hr_data = cleaned_phys_data[
                (cleaned_phys_data['Participant_ID'] == participant) &
                (cleaned_phys_data['Time_From_Start_Seconds'] >= window_start) &
                (cleaned_phys_data['Time_From_Start_Seconds'] <= window_end)
            ]

            gsr_data = cleaned_gsr_data[
                (cleaned_gsr_data['Participant_ID'] == participant) &
                (cleaned_gsr_data['Time_From_Start_Seconds'] >= window_start) &
                (cleaned_gsr_data['Time_From_Start_Seconds'] <= window_end)
            ]

            window_data = pd.concat([hr_data, gsr_data], axis=0).sort_values('Time_From_Start_Seconds')

            stats_dict = {
                'Participant_ID': participant,
                'Time_From_Start_Seconds': row['Window_Start_Second'],
                'Sample_Frame': sample_frame,
                'Condition': row['Condition']
            }

            # Rolling stats from all other columns
            if not window_data.empty:
                computed_stats = calculate_stats(
                                                    data          = window_data,
                                                    columns       = ALL_PHYS_COLUMNS,
                                                    participant_id= participant,
                                                    condition     = row['Condition'],
                                                    filter_value  = -1,
                                                    context       = "Rolling"
                                                )




                computed_stats = {f"Rolling_{key}": value for key, value in computed_stats.items()}
                stats_dict.update(computed_stats)
            else:
                logging.warning(f"[ROLLING] Empty window for Participant {participant}, Time {window_start_second}, skipping stats (NaNs will be inserted).")
                for col in ALL_PHYS_COLUMNS:
                    for stat in ['Median', 'Mean', 'SD', 'MIN', 'MAX']:
                        stats_dict[f"Rolling_{col}_{stat}"] = np.nan

            # HRV metrics in separate try/except
            rr_intervals = window_data['Polar_HeartRate_RR_Interval_CLEANED_ABS'].dropna()  # or group_data for full stats

            if len(rr_intervals) >= 2:
                hrv_metrics = extract_hrv_features(rr_intervals)
                rmssd    = hrv_metrics["HRV_RMSSD"].iloc[0]  if not pd.isna(hrv_metrics["HRV_RMSSD"]).all() else np.nan
                sdnn     = hrv_metrics["HRV_SDNN"].iloc[0]   if not pd.isna(hrv_metrics["HRV_SDNN"]).all() else np.nan
                pnn50    = hrv_metrics["HRV_pNN50"].iloc[0]  if not pd.isna(hrv_metrics["HRV_pNN50"]).all() else np.nan
            else:
                rmssd = sdnn = pnn50 = np.nan
            stats_dict.update({
                'Rolling_RMSSD': rmssd,
                'Rolling_SDNN': sdnn,
                'Rolling_pNN50': pnn50,
            })

            # Optional: flag HRV missing explicitly
            stats_dict['Rolling_HRV_Missing'] = int(np.isnan(rmssd))

            # Logging for empty rows
            if all(pd.isna(value) for key, value in stats_dict.items() if key.startswith("Rolling_")):
                logging.warning(f"[EMPTY STATS] All rolling stats NaN for P{participant}, Time {window_start_second}. Window rows: {len(window_data)}")
                logging.debug(f"Phys columns present: {window_data[ALL_PHYS_COLUMNS].notna().sum()}")

                # Print a summary of computed stats for this window
            print(f"[DEBUG] Computed Rolling Stats for P{participant}, Window starting at {window_start}:")
            for key, value in stats_dict.items():
                if key.startswith("Rolling_"):
                   print(f"    {key}: {value}")

                    
            results.append(stats_dict)

        except KeyError as e:
            print(f"[ERROR] KeyError at row {idx} — {e}")
            print(row)
            break


    rolling_stats_df = pd.DataFrame(results)

    present_pids = pd.DataFrame(results)['Participant_ID'].unique()
    print("[DEBUG] Participants included in rolling stats:", present_pids)

    eeg_and_phys_data = eeg_only_data.merge(
        rolling_stats_df, on=['Participant_ID', 'Sample_Frame', 'Condition'], how='left'
    )
    print("[INFO] Merged EEG and phys data shape:", eeg_and_phys_data.shape)
    return eeg_and_phys_data



###############################################
# Function: Calculate Full Condition Stats
###############################################

# --- Enforce datetime format for all time columns ---

if not np.issubdtype(cleaned_phys_data['Time_From_Start_Seconds'].dtype, np.datetime64):
    cleaned_phys_data['Time_From_Start_Seconds'] = pd.to_datetime('1970-01-01') + \
        pd.to_timedelta(cleaned_phys_data['Time_From_Start_Seconds'], unit='s')

if not np.issubdtype(cleaned_gsr_data['Time_From_Start_Seconds'].dtype, np.datetime64):
    cleaned_gsr_data['Time_From_Start_Seconds'] = pd.to_datetime('1970-01-01') + \
        pd.to_timedelta(cleaned_gsr_data['Time_From_Start_Seconds'], unit='s')

if not np.issubdtype(eeg_only_data['Window_Start_Second'].dtype, np.datetime64):
    eeg_only_data['Window_Start_Second'] = pd.to_datetime('1970-01-01') + \
        pd.to_timedelta(eeg_only_data['Window_Start_Second'], unit='s')

        
    

# NOTE:
# This function calculates rolling window statistics for physiological data by aligning it with EEG windows.
# For each EEG window, the 'Window_Start_Second' value is taken as the start of the window, and a 30-second window is defined.
# That is, physiologic metrics are aggregated over the time interval from the EEG window start to EEG window start + 30 seconds.
# This ensures that the physiological data are fully aligned with the EEG data if the EEG timestamp represents the window start.
# If, in another design, the EEG timestamps were to be interpreted as the midpoints of the windows,
# the window boundaries would need to be adjusted accordingly.

def calculate_full_condition_stats(cleaned_phys_data, cleaned_gsr_data, eeg_only_data):


    '''
    # Ensure time columns are datetime
    if not np.issubdtype(cleaned_phys_data['Time_From_Start_Seconds'].dtype, np.datetime64):
        cleaned_phys_data['Time_From_Start_Seconds'] = pd.to_datetime(cleaned_phys_data['Time_From_Start_Seconds'])
    if not np.issubdtype(eeg_only_data['Window_Start_Second'].dtype, np.datetime64):
        eeg_only_data['Window_Start_Second'] = pd.to_datetime(eeg_only_data['Window_Start_Second'])
    '''
    # Ensure Participant_ID is integer
    cleaned_phys_data['Participant_ID'] = cleaned_phys_data['Participant_ID'].astype('int64')
    eeg_only_data['Participant_ID'] = eeg_only_data['Participant_ID'].astype('int64')

    # ✅ derive the velocity time-series once for the whole recording
    cleaned_phys_data = extract_head_velocity_features(cleaned_phys_data)
    
    # Determine the active condition time range for each participant from EEG data.
    cond_time = eeg_only_data.groupby(['Participant_ID', 'Condition']).agg(
        condition_start=('Window_Start_Second_dt', 'min'),
        condition_end=('Window_Start_Second_dt', 'max')
    ).reset_index()

    # Step 1: Fix raw condition_end by adding 30s (to capture the real full duration)
    cond_time['condition_end'] = cond_time['condition_end'] + pd.Timedelta(seconds=30)

    # ================================
    # 📋 Step 2: Create Adjusted Windows to Focus on Middle of Condition
    # ================================

    # Why:
    # We don't want to use the first or last 30 seconds of a condition:
    # - First 30s = settling effects (physiological transition)
    # - Last 30s = anticipation artifacts (preparing for scene end)

    # This logic will only work properly for conditions that are about 3 minutes long 
    # (e.g., Relaxation and Stress scenes), which is what we are using for analysis now.

    # Step 2: Now create adjusted usable window (skip first and last 30s)
    cond_time['adjusted_condition_start'] = cond_time['condition_start'] + pd.Timedelta(seconds=0)
    cond_time['adjusted_condition_end'] = cond_time['condition_end'] - pd.Timedelta(seconds=0)

    # Step 3: Calculate adjusted window duration
    cond_time['adjusted_window_duration'] = (cond_time['adjusted_condition_end'] - cond_time['adjusted_condition_start']).dt.total_seconds()

    print(cond_time[['Participant_ID', 'Condition', 'adjusted_window_duration']])


    cond_time['window_duration'] = (cond_time['adjusted_condition_end'] - cond_time['adjusted_condition_start']).dt.total_seconds()
    print(cond_time[['Participant_ID', 'Condition', 'window_duration']])
    
    condition_stats_results = []

    cleaned_phys_data = cleaned_phys_data[
    cleaned_phys_data['Time_From_Start_Seconds'] >= cond_time['adjusted_condition_start'].min()
    ]

    
    for idx, row in tqdm(cond_time.iterrows(), total=len(cond_time), desc="Calculating Full Stats"):
        participant = row['Participant_ID']
        condition = row['Condition']
        start_time = row['adjusted_condition_start']
        end_time = row['adjusted_condition_end']

        try:
            hr_data = cleaned_phys_data[
                (cleaned_phys_data['Participant_ID'] == participant) &
                (cleaned_phys_data['Time_From_Start_Seconds'] >= start_time) &
                (cleaned_phys_data['Time_From_Start_Seconds'] <= end_time)
            ]

            gsr_data = cleaned_gsr_data[
                (cleaned_gsr_data['Participant_ID'] == participant) &
                (cleaned_gsr_data['Time_From_Start_Seconds'] >= start_time) &
                (cleaned_gsr_data['Time_From_Start_Seconds'] <= end_time)
            ]

            group_data = pd.concat([hr_data, gsr_data], axis=0).sort_values('Time_From_Start_Seconds')

        except KeyError:
            group_data = pd.DataFrame()


        stats_dict = {
            'Participant_ID': participant,
            'Condition': condition
        }

            # ✅ NEW: slice the *full* phys table for response metrics
        resp_window = cleaned_phys_data[
            (cleaned_phys_data['Participant_ID'] == participant) &
            (cleaned_phys_data['Time_From_Start_Seconds'] >= start_time) &
            (cleaned_phys_data['Time_From_Start_Seconds'] <= end_time)
        ]

        resp_metrics = extract_response_metrics(resp_window)
        stats_dict.update({f"Full_{k}": v for k, v in resp_metrics.items()})

        # --- HRV ---
        rr_intervals = group_data['Polar_HeartRate_RR_Interval_CLEANED_ABS'].dropna()

        if len(rr_intervals) >= 2:
            hrv_metrics = extract_hrv_features(rr_intervals)
            # NeuroKit returns:
            #   HRV_RMSSD in seconds,
            #   HRV_SDNN in milliseconds,
            #   HRV_pNN50 in percent.
            rmssd_s  = hrv_metrics["HRV_RMSSD"].iloc[0]
            sdnn     = hrv_metrics["HRV_SDNN"].iloc[0]
            pnn50    = hrv_metrics["HRV_pNN50"].iloc[0]

            # convert RMSSD → ms
            rmssd_ms = rmssd_s * 1000

        else:
            rmssd_ms = sdnn = pnn50 = np.nan

        # write them once, using the ms version for RMSSD
        stats_dict.update({
            'Full_RMSSD':  rmssd_ms,
            'Full_SDNN':   sdnn,
            'Full_pNN50':  pnn50,
        })


        # --- Other stats ---
        if not group_data.empty:
            computed_stats = calculate_stats(group_data, ALL_PHYS_COLUMNS, participant, condition, filter_value=-1, context="Full")
            computed_stats = {f"Full_{key}": value for key, value in computed_stats.items()}
            stats_dict.update(computed_stats)
        else:
            for col in ALL_PHYS_COLUMNS:
                for stat in ['Median', 'Mean', 'SD', 'MIN', 'MAX']:
                    stats_dict[f"Full_{col}_{stat}"] = np.nan

        # --- Baseline values ---
        for base in BASELINE_MEASURES:
            stats_dict[f"Full_{base}"] = group_data[base].mean() if base in group_data.columns and not group_data.empty else np.nan

        # Save row
        condition_stats_results.append(stats_dict)

    # ✅ Now create the DataFrame **once** outside the loop
    condition_stats_df = pd.DataFrame(condition_stats_results)
    return condition_stats_df




=== Columns in eeg_only_data ===
['Participant_ID', 'Window_Start_Second', 'Condition', 'Sample_Frame', 'FrontalLeft_Delta_Power', 'FrontalLeft_Theta_Power', 'FrontalLeft_LowAlpha_Power', 'FrontalLeft_HighAlpha_Power', 'FrontalLeft_LowBeta_Power', 'FrontalLeft_HighBeta_Power', 'FrontalLeft_Alpha_Power', 'FrontalLeft_Beta_Power', 'FrontalRight_Delta_Power', 'FrontalRight_Theta_Power', 'FrontalRight_LowAlpha_Power', 'FrontalRight_HighAlpha_Power', 'FrontalRight_LowBeta_Power', 'FrontalRight_HighBeta_Power', 'FrontalRight_Alpha_Power', 'FrontalRight_Beta_Power', 'OverallFrontal_Delta_Power', 'OverallFrontal_Theta_Power', 'OverallFrontal_LowAlpha_Power', 'OverallFrontal_HighAlpha_Power', 'OverallFrontal_LowBeta_Power', 'OverallFrontal_HighBeta_Power', 'OverallFrontal_Alpha_Power', 'OverallFrontal_Beta_Power', 'Temporal_Delta_Power', 'Temporal_Theta_Power', 'Temporal_LowAlpha_Power', 'Temporal_HighAlpha_Power', 'Temporal_LowBeta_Power', 'Temporal_HighBeta_Power', 'Temporal_Alpha_Power', 'Te

In [ ]:
# Assume eeg_only_data is already loaded from your EEG feature extraction pipeline.
# We'll start by making a copy of it as our base for merging with phys stats.
eeg_and_phys_data_cleaned = eeg_only_data.copy()

'''

# --- Execute Rolling Stats Calculation ---
print("[INFO] Starting rolling stats calculation...")
rolling_stats_df = calculate_rolling_stats_all(cleaned_phys_data, cleaned_gsr_data, eeg_only_data)

# Ensure merge keys are of the same type (convert Participant_ID to int)
eeg_and_phys_data_cleaned['Participant_ID'] = eeg_and_phys_data_cleaned['Participant_ID'].astype(int)
rolling_stats_df['Participant_ID'] = rolling_stats_df['Participant_ID'].astype(int)

# If there are any existing Rolling_ columns in the base DataFrame, drop them to avoid duplication
rolling_cols = [col for col in eeg_and_phys_data_cleaned.columns if col.startswith("Rolling_")]
if rolling_cols:
    eeg_and_phys_data_cleaned = eeg_and_phys_data_cleaned.drop(columns=rolling_cols)

# Merge rolling stats on 'Participant_ID', 'Sample_Frame', and 'Condition'
eeg_and_phys_data_cleaned = eeg_and_phys_data_cleaned.merge(
    rolling_stats_df,
    on=['Participant_ID', 'Sample_Frame', 'Condition'],
    how='left'
)
print("[INFO] Rolling stats merged successfully. Data shape:", eeg_and_phys_data_cleaned.shape)

'''

# --- Execute Full Condition Stats Calculation ---
print("[INFO] Starting full condition stats calculation...")
condition_stats_df = calculate_full_condition_stats(cleaned_phys_data, cleaned_gsr_data, eeg_only_data)

# Ensure merge keys are of the same type
eeg_and_phys_data_cleaned['Participant_ID'] = eeg_and_phys_data_cleaned['Participant_ID'].astype(int)
condition_stats_df['Participant_ID'] = condition_stats_df['Participant_ID'].astype(int)

# Only drop the Full_ columns that are in condition_stats_df
full_cols_to_drop = [col for col in condition_stats_df.columns if col.startswith("Full_")]
existing_cols_to_drop = [col for col in full_cols_to_drop if col in eeg_and_phys_data_cleaned.columns]

if existing_cols_to_drop:
    print(f"[INFO] Dropping overlapping Full_ columns from base DataFrame: {len(existing_cols_to_drop)}")
    eeg_and_phys_data_cleaned = eeg_and_phys_data_cleaned.drop(columns=existing_cols_to_drop)


# Merge full condition stats on 'Participant_ID' and 'Condition'
eeg_and_phys_data_cleaned = eeg_and_phys_data_cleaned.merge(
    condition_stats_df,
    on=['Participant_ID', 'Condition'],
    how='left'
)
print("[INFO] Full condition stats merged successfully. Final merged data shape:", eeg_and_phys_data_cleaned.shape)


# Print current columns for reference
print("Columns before cleaning:")
print(eeg_and_phys_data_cleaned.columns.tolist())

# --- Convert the chosen time column to seconds ---
# Here, we assume that after merging, the seconds version is in "Window_Start_Second_x"
if 'Window_Start_Second_x' in eeg_and_phys_data_cleaned.columns:
    # Rename it to "Time"
    eeg_and_phys_data_cleaned.rename(columns={'Window_Start_Second_x': 'Time'}, inplace=True)
else:
    print("[WARNING] 'Window_Start_Second_x' not found.")

# Drop the original "Time" column if you don't need it:
if 'Time' in eeg_and_phys_data_cleaned.columns:
    eeg_and_phys_data_cleaned.drop(columns='Time', inplace=True)
else:
    print("[INFO] Column 'Time' not present – nothing to drop.")


# --- Remove duplicate EEG feature columns ---
# For instance, if merging produced columns like "Parietal_Theta_Power_x" and "Parietal_Theta_Power_y",
# and you want to keep only the _x version:
cols_to_drop = [col for col in eeg_and_phys_data_cleaned.columns if col.endswith('_y')]
if cols_to_drop:
    print("[INFO] Dropping duplicate columns:", cols_to_drop)
    eeg_and_phys_data_cleaned.drop(columns=cols_to_drop, inplace=True)

# Then, remove the '_x' suffix from the remaining columns for clarity:
eeg_and_phys_data_cleaned.columns = [col.replace('_x', '') for col in eeg_and_phys_data_cleaned.columns]

# --- Final Check ---
print("Final column names:")
print(eeg_and_phys_data_cleaned.columns.tolist())



[INFO] Starting full condition stats calculation...
     Participant_ID                     Condition  adjusted_window_duration
0                 1       HighStress_HighCog_Task                     180.0
1                 1        HighStress_LowCog_Task                     180.0
2                 1        LowStress_HighCog_Task                     180.0
3                 1         LowStress_LowCog_Task                     180.0
4                 1  Post_Exposure_Blank_Baseline                      60.0
..              ...                           ...                       ...
561              48    Pre_Exposure_Room_Baseline                      60.0
562              48                   Relaxation1                     180.0
563              48                   Relaxation2                     180.0
564              48                   Relaxation3                     180.0
565              48                   Relaxation4                     180.0

[566 rows x 3 columns]
     Partici

Calculating Full Stats:   0%|          | 0/566 [00:00<?, ?it/s]

🔍 extract_hrv_features called
1) Raw input length: 4447
   After NaN removal: 4447 values; head/tail: [642. 642. 642. 642. 642.] ... [785. 785. 785. 785. 785.]
2) RR stats (ms): min=549.0, max=899.0, mean=719.37, std=96.50
3) Converted to seconds; head/tail: [0.642 0.642 0.642 0.642 0.642] ... [0.785 0.785 0.785 0.785 0.785]
4) RRI dict prepared; first 5 times: [0.642 1.284 1.926 2.568 3.21 ]
5) Computing time-domain HRV with nk.hrv_time()
   HRV DataFrame:
   HRV_MeanNN  HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0     0.71937  0.096507         NaN         NaN         NaN         NaN   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD  HRV_SDSD  ...  HRV_IQRNN  HRV_SDRMSSD  \
0         NaN         NaN   0.006154  0.006155  ...      0.175    15.681442   

   HRV_Prc20NN  HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN  \
0        0.594        0.805        0.0        0.0      0.549      0.899   

   HRV_HTI  HRV_TINN  
0      1.0       0.0  

[1 rows x 25 columns]
6) Retur

C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 900 is greater than input length  = 800, using nperseg = 800
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 900 is greater than input length  = 800, using nperseg = 800
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 900 is greater than input length  = 800, using nperseg = 800
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
C:\Users\Joe\AppDat

🔍 extract_hrv_features called
1) Raw input length: 1329
   After NaN removal: 1329 values; head/tail: [797. 797. 797. 797. 797.] ... [755. 755. 755. 755. 755.]
2) RR stats (ms): min=755.0, max=890.0, mean=843.23, std=27.78
3) Converted to seconds; head/tail: [0.797 0.797 0.797 0.797 0.797] ... [0.755 0.755 0.755 0.755 0.755]
4) RRI dict prepared; first 5 times: [0.797 1.594 2.391 3.188 3.985]
5) Computing time-domain HRV with nk.hrv_time()
   HRV DataFrame:
   HRV_MeanNN  HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0    0.843226  0.027792         NaN         NaN         NaN         NaN   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD  HRV_SDSD  ...  HRV_IQRNN  HRV_SDRMSSD  \
0         NaN         NaN   0.004251  0.004252  ...      0.038     6.538416   

   HRV_Prc20NN  HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN  \
0        0.821        0.864        0.0        0.0      0.755       0.89   

   HRV_HTI  HRV_TINN  
0      1.0       0.0  

[1 rows x 25 columns]
6) Retur

...

🔍 extract_hrv_features called
1) Raw input length: 4490
   After NaN removal: 4490 values; head/tail: [ 975.         1028.         1028.         1028.         1025.37887442] ... [876. 876. 876. 876. 876.]
2) RR stats (ms): min=-1521.813163317082, max=1633.821448473554, mean=770.56, std=214.43
3) Converted to seconds; head/tail: [0.975      1.028      1.028      1.028      1.02537887] ... [0.876 0.876 0.876 0.876 0.876]
4) RRI dict prepared; first 5 times: [0.975      2.003      3.031      4.059      5.08437887]
5) Computing time-domain HRV with nk.hrv_time()
   HRV DataFrame:
   HRV_MeanNN  HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0     0.77056  0.214457         NaN         NaN         NaN         NaN   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD  HRV_SDSD  ...  HRV_IQRNN  HRV_SDRMSSD  \
0         NaN         NaN   0.692328       NaN  ...      0.217     0.309761   

   HRV_Prc20NN  HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN  \
0        0.597        0.904     

C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


🔍 extract_hrv_features called
1) Raw input length: 5181
   After NaN removal: 5181 values; head/tail: [928. 928. 928. 928. 928.] ... [848. 848. 848. 848. 848.]
2) RR stats (ms): min=264.0, max=1705.3959998222233, mean=795.45, std=106.28
3) Converted to seconds; head/tail: [0.928 0.928 0.928 0.928 0.928] ... [0.848 0.848 0.848 0.848 0.848]
4) RRI dict prepared; first 5 times: [0.928 1.856 2.784 3.712 4.64 ]
5) Computing time-domain HRV with nk.hrv_time()
   HRV DataFrame:
   HRV_MeanNN  HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0    0.795448  0.106286         NaN         NaN         NaN         NaN   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD  HRV_SDSD  ...  HRV_IQRNN  HRV_SDRMSSD  \
0         NaN         NaN   0.024929  0.024931  ...      0.099     4.263582   

   HRV_Prc20NN  HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN  \
0        0.752        0.872        0.0        0.0      0.264   1.705396   

   HRV_HTI  HRV_TINN  
0      1.0       0.0  

[1 rows x 25 col

C:\Users\Joe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


🔍 extract_hrv_features called
1) Raw input length: 3971
   After NaN removal: 3971 values; head/tail: [850. 850. 850. 850. 850.] ... [755. 755. 755. 755. 755.]
2) RR stats (ms): min=-687.4070767171343, max=1388.0, mean=687.27, std=150.03
3) Converted to seconds; head/tail: [0.85 0.85 0.85 0.85 0.85] ... [0.755 0.755 0.755 0.755 0.755]
4) RRI dict prepared; first 5 times: [0.85 1.7  2.55 3.4  4.25]
5) Computing time-domain HRV with nk.hrv_time()
   HRV DataFrame:
   HRV_MeanNN  HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0    0.687265  0.150054         NaN         NaN         NaN         NaN   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD  HRV_SDSD  ...  HRV_IQRNN  HRV_SDRMSSD  \
0         NaN         NaN   0.051295  0.051301  ...       0.12      2.92532   

   HRV_Prc20NN  HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN  \
0        0.611        0.762        0.0        0.0  -0.687407      1.388   

    HRV_HTI  HRV_TINN  
0  1.001261       0.0  

[1 rows x 25 columns]
6

# Cell 7: Merging & Final Export

In [ ]:
# ---- 1. Load & Reshape Subjective Data ----
print("starting cell")


# Load the data
if not os.path.exists(subjective_file_path):
    raise FileNotFoundError(f"Subjective data file not found at: {subjective_file_path}")

subjective_df = pd.read_csv(subjective_file_path)

# Mapping for reshaping subjective data (rename to avoid conflict later)
subjective_condition_map = {
    'Calm Addition': 'LowStress_LowCog_Task',
    'Calm Subtraction': 'LowStress_HighCog_Task',
    'Stress Addition': 'HighStress_LowCog_Task',
    'Stress Subtraction': 'HighStress_HighCog_Task'
}


def map_subjective_condition(task_type):
    return subjective_condition_map.get(task_type, task_type)

# Reshape the subjective data into a dictionary format
reshaped_dict = {}
for col in subjective_df.columns[1:]:
    task_type, metric = col.rsplit(' ', 1)
    condition = map_subjective_condition(task_type)
    for _, row in subjective_df.iterrows():
        # Convert Participant_ID to an integer-based string for consistency
        participant = str(int(float(row['Participant_ID'])))
        key = (participant, condition)
        if key not in reshaped_dict:
            reshaped_dict[key] = {'Participant_ID': participant, 'Condition': condition}
        reshaped_dict[key][metric] = row[col]

# Convert the reshaped data into a DataFrame
subjective_data = pd.DataFrame.from_dict(reshaped_dict, orient='index')
subjective_data.reset_index(drop=True, inplace=True)

# ---- 2. Clean EEG & Phys Data ----
# (Assuming eeg_and_phys_data is already loaded)
if 'Participant' in eeg_and_phys_data_cleaned.columns:
    eeg_and_phys_data_cleaned.rename(columns={'Participant': 'Participant_ID'}, inplace=True)

# Ensure Participant_ID is a clean string in both dataframes
eeg_and_phys_data_cleaned['Participant_ID'] = eeg_and_phys_data_cleaned['Participant_ID'].astype(str).str.strip()
subjective_data['Participant_ID'] = subjective_data['Participant_ID'].astype(str).str.strip()

## Check unique Participant_IDs and Conditions in both datasets
print("[DEBUG] Unique Participant IDs in EEG/Phys Data:", eeg_and_phys_data_cleaned['Participant_ID'].unique())
print("[DEBUG] Unique Participant IDs in Subjective Data:", subjective_data['Participant_ID'].unique())

print("[DEBUG] Unique Conditions in EEG/Phys Data:", eeg_and_phys_data_cleaned['Condition'].unique())
print("[DEBUG] Unique Conditions in Subjective Data:", subjective_data['Condition'].unique())

# Count missing (NaN) values for each column
missing_values = eeg_and_phys_data_cleaned.isna().sum()

# Filter to show only columns with missing values
missing_values = missing_values[missing_values > 0]

# Sort by the number of missing values (optional for better visibility)
missing_values = missing_values.sort_values(ascending=False)

# Print the missing values summary
print("[INFO] Missing Values in eeg_and_phys_data_cleaned:")
for col, missing_count in missing_values.items():
    print(f" - {col}: {missing_count} missing values")



# ---- 3. Merge EEG/Phys Data with Subjective Data ----
final_with_subjective = pd.merge(
    eeg_and_phys_data_cleaned,
    subjective_data,
    left_on=['Participant_ID', 'Condition'],   # Use 'Condition' from EEG data
    right_on=['Participant_ID', 'Condition'],  # Use 'Mapped_Condition' from Subjective data
    how='left'
)

# List of Baseline Measures
BASELINE_MEASURES = [
    'Median_0_RestingHR_Baseline',
    'Mean_0_RestingHR_Baseline',
    'SD_0_RestingHR_Baseline',
    'Median_1_RestingHR_Interval_Baseline',
    'Mean_1_RestingHR_Interval_Baseline',
    'SD_1_RestingHR_Interval_Baseline',
    'Median_0_GSRConductance_Baseline',
    'Mean_0_GSRConductance_Baseline',
    'SD_0_GSRConductance_Baseline',
    'Median_1_GSRResistance_Baseline',
    'Mean_1_GSRResistance_Baseline',
    'SD_1_GSRResistance_Baseline'
]


# Debug: Final merged data shape and sample rows
print("[DEBUG] Final Merged Data Shape:", final_with_subjective.shape)
print("[DEBUG] Sample Data:\n", final_with_subjective.head())


# ---- 4. Save Final Aggregated Data ----
final_output_path = os.path.join(config["paths"]["output"], "aggregated", "all_data_aggregated.csv")
final_with_subjective.to_csv(final_output_path, index=False)
print(f"[INFO] Merge completed successfully. File saved as: {final_output_path}")


starting cell
[DEBUG] Unique Participant IDs in EEG/Phys Data: ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48']
[DEBUG] Unique Participant IDs in Subjective Data: ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48']
[DEBUG] Unique Conditions in EEG/Phys Data: ['Pre_Exposure_Blank_Baseline' 'Pre_Exposure_Room_Baseline' 'Relaxation1'
 'HighStress_HighCog_Task' 'Relaxation2' 'HighStress_LowCog_Task'
 'Relaxation3' 'LowStress_HighCog_Task' 'Relaxation4'
 'LowStress_LowCog_Task' 'Post_Exposure_Blank_Baseline'
 'Post_Exposure_Room_Baseline']
[DEBUG] Unique Conditions in Subjective Data: ['LowStress_LowCog_Task' 'LowStress_HighCo